# Import

In [58]:
import csv
import os
import sys

import numpy as np
import pandas as pd
import umap
from scipy.sparse.csgraph import connected_components
from sklearn.decomposition import PCA
from tqdm import tqdm

## Add configuration file

In [37]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [38]:
from ALL import config 
from util import *

## Set condition

In [39]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [40]:
data_type="AgNews"

# Read data

In [41]:
df = pd.read_csv(
    f"../../Preprocessing/data/{data_type}/master.csv", index_col=0
)

In [43]:
with open(f"../../Preprocessing/data/{data_type}/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [44]:
max_model_num = config["vectorize"]["sentenceBERT"]["max_model_num"]
vector_dims = config["vectorize"]["sentenceBERT"]["dims"]

# Dimension Depression

In [64]:
vector_path = f"../data/{data_type}/sentenceBERT/vector"

for vector_dim in tqdm(vector_dims):
    for model_num in range(max_model_num):
        vectors = np.load(
            f"{vector_path}/raw/{model_num}.npy",
        )
        reduced_vectors = PCA(
            n_components=vector_dim, random_state=model_num
        ).fit_transform(vectors)

        np.save(
            make_filepath(f"{vectors_path}/pca/{model_num}.npy"),
            reduced_vectors,
        )

100%|██████████| 9/9 [00:00<00:00, 17.88it/s]


In [65]:
vector_path = f"../data/{data_type}/sentenceBERT/vector"

for vector_dim in tqdm(vector_dims):
    for model_num in range(max_model_num):
        vectors = np.load(
            f"{vector_path}/raw/{model_num}.npy",
        )
        reduced_vectors = umap.UMAP(
            n_components=vector_dim, random_state=model_num
        ).fit_transform(vectors)
        np.save(
            make_filepath(f"{vectors_path}/umap/{model_num}.npy"),
            reduced_vectors,
        )

100%|██████████| 9/9 [00:54<00:00,  6.05s/it]


In [17]:
# import shutil
# shutil.rmtree('data/AgNews/sentenceBERT/')